In [149]:
 require 'rest-client'

  def fetch(url, headers = {accept: "*/*"}, user = "", pass="")
  
    response = RestClient::Request.execute({
      method: :get,
      url: url.to_s,
      user: user,
      password: pass,
      headers: headers})

  return response
  
  rescue RestClient::ExceptionWithResponse => e
    $stderr.puts e.inspect
    response = false
    return response  
  rescue RestClient::Exception => e
    $stderr.puts e.inspect
    response = false
    return response  
  rescue Exception => e
    $stderr.puts e.inspect
    response = false
    return response  
    
end 

:fetch

In [150]:
class ListNetworks
  
  @@all_interactions = Array.new
  attr_accessor :network_ID
  attr_accessor :interactors
  attr_accessor :network_go
  attr_accessor :network_kegg
  
  def initialize (params = {})
    @network_ID = params.fetch(:network_ID, "000000000")
    @interactors = params.fetch(:interactors, Array.new)
    @network_go = params.fetch(:network_go, Array.new)
    @network_kegg = params.fetch(:network_kegg, Array.new)
    
    @@all_interactions << self
    
  end
    
  def ListNetworks.is_in_the_list(gene_to_check) 
    if $list_of_genes.include?("#{gene_to_check}") 
      if @network_ID == nil
        ListNetworks.new(:network_ID => $AGI)
        @interactors |= [gene_to_check]
        @network_go |= [InteractionNetwork.find_go(gene_to_check)]
        @network_kegg |= [InteractionNetwork.find_kegg(gene_to_check)]
      elsif @network_ID.include?($AGI) and (not @interactors.include?(gene_to_check))
        @interactors << [gene_to_check]
        @network_go << InteractionNetwork.find_go(gene_to_check)
        @network_kegg << InteractionNetwork.find_kegg(gene_to_check)
      else
        ListNetworks.new(:network_ID => $AGI)
        @interactors << [gene_to_check]
        @network_go << InteractionNetwork.find_go(gene_to_check)
        @network_kegg << InteractionNetwork.find_kegg(gene_to_check)
      end
    end
  end
end

:is_in_the_list

In [151]:
class InteractionNetwork
  
  attr_accessor :gene_ID  
  attr_accessor :interactions
  attr_accessor :go_ID
  attr_accessor :kegg_pathways
  
  
  def initialize (params = {})
    
    @gene_ID = params.fetch(:gene_ID, "000000000")
    @interactions = params.fetch(:interactions, Array.new)
    @go_bio = params.fetch(:go_bio, Array.new)
    @kegg_pathways = params.fetch(:kegg_pathways, Array.new)
    
    $AGI = @gene_ID
    
  end
  
  def InteractionNetwork.is_agi
    return @gene_ID
  end
  
  def InteractionNetwork.find_go(gene1)
    res = fetch("http://togows.org/entry/ebi-uniprot/#{gene1}/dr.json");
    if res
      data = JSON.parse(res.body)
      if data[0] and data[0]["GO"]
        final_go = []
      data[0]["GO"].each do |go|
        if go[1].match(/P:/)
          final_go << [go[0]+ go[1]]
          return final_go
        end
      end
    end
    else
      puts "togows.org failed - see STDERR for details..."
    end
  end
  
  def InteractionNetwork.find_kegg(gene2)
    ros = fetch("http://togows.org/entry/kegg-genes/ath:#{gene2}/pathways.json");
    if ros
      data = JSON.parse(ros.body)
      if data[0]
        final_kegg = []
      data[0].each do |kegg|
        final_kegg << [kegg[0]+":"+kegg[1]]
        return final_kegg 
      end
    end
    else
      puts "togows.org failed - see STDERR for details..."
    end
  end
  
  def search_interactions(some_gene, depth) # Function to search the interaction
    if depth <= 2
      depth = depth + 1
      link = fetch("http://bar.utoronto.ca:9090/psicquic/webservices/current/search/query/#{some_gene}"); # Link to search for interctions indicated in http://bar.utoronto.ca/affydb/BAR_instructions.html#AIV
      if link 
        body = link.to_s.split("\n")
        body.each do |interaction|
          line = interaction.split("\t")
          agi1 = line[2].split(":")[1].upcase
          agi2 = line[3].split(":")[1].upcase
          tax1 = line[9].split(":")[1]
          tax2 = line[10].split(":")[1]
          score = line[14].split(":")[1]
          if tax1.to_i != 3702 or tax2.to_i !=3702 # Filtering by species (Arabidopsis thaliana).
            next
          elsif score.to_f < 0.45 # Filtering by an intermediate score (DOI: 10.1093/database/bau131).
            next
          elsif some_gene.to_s == agi1.to_s and some_gene.to_s == agi2.to_s # Discarding interactions beteen the same gene.         
            next
          elsif not (agi1.match(/AT\dG\d\d\d\d\d/)) or not (agi2.match(/AT\dG\d\d\d\d\d/))
            next
          else
            case # Checking the position of the query gene in the "interaction vector"
            when agi1.to_s == some_gene.to_s 
              ggi = [some_gene, agi2]
              ggi_inverse = [agi2, some_gene]
              if not (@interactions.include?(ggi)) and not (@interactions.include?(ggi_inverse)) and agi2 != @gene_ID # Checking that the interaction list does not include this interaction
                @interactions << ggi 
                @go_bio << InteractionNetwork.find_go(agi2)
                @kegg_pathways << InteractionNetwork.find_kegg(agi2)
                ListNetworks.is_in_the_list(agi2)
                search_interactions(agi2, depth) # Applying recursivity to search for interactions of the genes linked to the input genes
              end
            else
              ggi = [some_gene, agi1]
              ggi_inverse = [agi1, some_gene]
              if not (@interactions.include?(ggi)) and not (@interactions.include?(ggi_inverse)) and agi1 != @gene_ID # Checking that the interaction list does not include this interaction
                @interactions << ggi #  podríamos hacer un flatten! y un uniq
                @go_bio << InteractionNetwork.find_go(agi1)
                @kegg_pathways << InteractionNetwork.find_kegg(agi1)
                ListNetworks.is_in_the_list(agi1)
                search_interactions(agi1, depth) # Applying recursivity to search for interactions of the genes linked to the input genes
              end
            end
          end
        end
      else
        puts "bar.utoronto.ca call failed - see STDERR for details..."
    end
  end
end
end

:search_interactions

In [152]:
$list_of_genes = []
input = File.open("Arabidopsis_prueba2.txt", "r").each do |gene_id|
  $list_of_genes << gene_id.strip.upcase
  genes = InteractionNetwork.new(:gene_ID => gene_id.strip.upcase)
  genes.search_interactions(genes.gene_ID,1)
end
input.close

output = File.open("Output.txt", mode: "w")
output << "Assignment 2\nLists of input genes networks that interact with other input genes:\n"
output.close

add = File.open("Output.txt", "a")
$all_interactions.each do |network|
  add.puts network
end
add.close